In [1]:
from get_data import ceramiche_DB
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.model_selection import (GridSearchCV, cross_validate,
                                     train_test_split)
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from tqdm import tqdm


In [2]:
X, y = ceramiche_DB()
nn = MLPClassifier(max_iter=500, solver='adam')
params = dict(
    hidden_layer_sizes  = list(range(2,30)),
    activation          = ['identity', 'logistic', 'tanh', 'relu']
)
clf = GridSearchCV(estimator=nn, param_grid=params, cv=5, scoring='accuracy', verbose = 0, n_jobs = -1)

In [3]:
def sensibility(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[1][1]/sum(c_matrix[1])


def specificity(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[0][0]/sum(c_matrix[0])


scores = {
    'sensibility' : make_scorer(sensibility, greater_is_better=True),
    'specificity' : make_scorer(specificity, greater_is_better=True),
    'accuracy' : make_scorer(accuracy_score, greater_is_better=True)
}

In [4]:
import warnings
from time import process_time
t = process_time()
with warnings.catch_warnings():  
    scores = cross_validate(clf, X, y,
                            cv=5, return_estimator=True,
                            scoring = scores )
process_time-t

C:\Users\scutt\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\scutt\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\scutt\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\scutt\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will 

In [11]:
rows = ['sensibility', 'specificity', 'accuracy']
columns = ['mean', 'std']

row_format ="{:>20}" * (len(columns) + 1)
col_format ="{:>20}" * (len(columns) + 1)

data = [(scores["test_"+x].mean(), scores["test_"+x].std()) for x in rows]


print(col_format.format("", *columns))
for name, row in zip(rows, data):
    print(row_format.format(name, *row))


                                    mean                 std
         sensibility  0.9529411764705882 0.09411764705882356
         specificity  0.8714285714285716 0.19378085666072195
            accuracy   0.931060606060606 0.08744669109398835
